In [ ]:
import ROOT
import glob
from datetime import datetime
import math
from copy import copy
%jsroot on

ROOT.gStyle.SetGridStyle(3)
ROOT.gStyle.SetGridWidth(1)
ROOT.gStyle.SetOptStat(0)
ROOT.gStyle.SetPalette(57)
palette = list(ROOT.TColor.GetPalette())

In [ ]:
futures[0]

In [ ]:
path = "/cave/alice-tpc-qc/data/2023/LHC23r/apass3/536606.root"
file = ROOT.TFile(path,"r")

In [ ]:
path = "/cave/alice-tpc-qc/data/2023/LHC23r/apass3/536606.root"
file = ROOT.TFile(path,"r")
with ROOT.TFile(path.split(".root")[0]+"_QC.root", "update") as outfile:
    outfile.cd()
    for folder in file.mw.TPC.GetListOfKeys():
        timestamps = [timestamp.GetName() for timestamp in file.mw.TPC.Get(folder.GetName()).GetListOfKeys()]
        objects = [obj.GetName() for obj in file.mw.TPC.Get(folder.GetName()).Get(timestamps[0])]
        if not len(timestamps) : continue
        outfile.cd(folder.GetName()+"QC")
        for item in range(len(objects)):
            ROOT.gDirectory.mkdir(objects[item]+"_mw")
            outfile.cd(folder.GetName()+"QC/"+objects[item]+"_mw")
            for timestamp in timestamps:
                histo = file.mw.TPC.Get(folder.GetName()).Get(timestamp).At(item).getObject()
                histo.SetName(timestamp)
                histo.Write("")
                #outfile.WriteObject(hout, "myhisto")

In [ ]:
#file = ROOT.TFile("/cave/alice-tpc-qc/data/2023/LHC23r/apass3/test.root","RECREATE")
ROOT.TFile("/cave/alice-tpc-qc/data/2023/LHC23r/apass3/test.root","a")
with ROOT.TFile(path.split(".root")[0]+"_QC.root", "recreate") as outfile:
    outfile.cd()
    ROOT.gDirectory.mkdir("PID")
    outfile.cd("PID")
    histo.Write("")

In [ ]:
timestamps = [timestamp.GetName() for timestamp in file.mw.TPC.PID.GetListOfKeys()]
pidObjects = [obj.GetName() for obj in file.mw.TPC.PID.Get(timestamps[0])]
tracksObjects = [obj.GetName() for obj in file.mw.TPC.Tracks.Get(timestamps[0])]

In [ ]:
histograms = []
for task in ["PID"]:
    timestamps = [timestamp.GetName() for timestamp in file.mw.TPC.Get(task).GetListOfKeys()]
    myObjects = [obj.GetName() for obj in file.mw.TPC.Get(task).Get(timestamps[0])]
    for timestamp in timestamps:
        for item in range(len(pidObjects)):
            histo = file.mw.TPC.Get("PID").Get(timestamp).At(item).getObject()
            histo.SetName(timestamp)
            histo.Scale(1/histo.Integral())
            histograms.append(histo)

In [ ]:
def drawMovingWindowOverlay(histograms,timestamps):
    canvas = ROOT.TCanvas()
    canvas.SetGrid()
    leg = ROOT.TLegend()
    leg.SetHeader(datetime.fromtimestamp(int(timestamps[0])/1000).strftime('%d-%b-%y'))
    leg.SetNColumns(round(len(timestamps)/6))
    for i,hist in enumerate(histograms):
        color = palette[math.floor((i/(len(histograms)-1))*(len(palette)-1))]
        hist.SetLineColor(color)
        hist.SetMarkerColor(color)
        hist.Draw("SAME L")
        leg.AddEntry(hist,datetime.fromtimestamp(int(hist.GetName())/1000).strftime('%H:%M'))
    return canvas,leg

def drawMovingWindow2D(histograms, timestamps):
    timestep = (int(timestamps[1])-int(timestamps[0]))/2
    my2D = ROOT.TH2F(histograms[0].GetName()+"_mw",
                     histograms[0].GetTitle() +" Moving Window",
                     len(histograms),int(timestamps[0])-timestep,int(timestamps[-1])+timestep,
                     histograms[0].GetNbinsX(),
                     histograms[0].GetXaxis().GetXmin(),
                     histograms[0].GetXaxis().GetXmax())
    for i,hist in enumerate(histograms):
        for binx in range(hist.GetNbinsX()):
            my2D.Fill(int(timestamps[i]),hist.GetBinCenter(binx+1),hist.GetBinContent(binx))
    canvas = ROOT.TCanvas()
    my2D.GetXaxis().SetTimeDisplay(1)
    my2D.GetXaxis().SetTitle("Time")
    my2D.GetYaxis().SetTitle(histograms[0].GetXaxis().GetTitle())
    my2D.Draw("COLZ")
    return my2D,canvas

In [ ]:
canvas,leg = drawMovingWindowOverlay(histograms,timestamps)
leg.Draw()
canvas.Draw()

In [ ]:
hist2D

In [ ]:
with ROOT.TFile("~/Desktop/file1.root", "recreate") as outfile:
    outfile.WriteObject(hist2D, "myHIst")

In [ ]:
timestamps = [int(val)/1000 for val in timestamps]

In [ ]:
timestep = (int(timestamps[1])-int(timestamps[0]))/2
my2d = ROOT.TH2F("","",len(histograms),int(timestamps[0])-timestep,int(timestamps[-1])+timestep,
                 histograms[0].GetNbinsX(),
                 histograms[0].GetXaxis().GetXmin(),
                 histograms[0].GetXaxis().GetXmax())
for i,hist in enumerate(histograms):
    for binx in range(hist.GetNbinsX()):
        my2d.Fill(int(timestamps[i]),hist.GetBinCenter(binx+1),hist.GetBinContent(binx))
canvas = ROOT.TCanvas()
my2d.GetXaxis().SetTimeDisplay(1)
my2d.GetXaxis().SetTitle("Time")
my2d.GetYaxis().SetTitle(histograms[0].GetXaxis().GetTitle())
my2d.Draw("COLZ")
my2d.SetTitle("Moving Windows - "+histograms[0].GetTitle())
myFunc = ROOT.TF1("myFunc","gaus",40,60)
my2d.FitSlicesY(myFunc, 0, -1, 0, "QNR")
fit = copy(ROOT.gDirectory.Get(my2d.GetName()+"_1"))
fit.SetLineWidth(2)
fit.SetMarkerStyle(21)
fit.SetMarkerSize(0.6)
fit.Draw("SAME P E")
canvas.Draw()

In [ ]:
datetime.fromtimestamp(1697089439887/1000).strftime('%H:%M %d-%b-%Y')

In [ ]:
file.mw.TPC.Get("PID")